In [1]:
str1 = "Karachi is one of the most congested city in the world."
str2 = "Karachi is the financial capital of Pakistan."
str3 = "Islamabad is the capital of Pakistan."

In [2]:
strlist = [str1,str2,str3]
stopword = set("of the a is in".split())
print(stopword)
s = []
for str in strlist: 
    s_list = [word for word in str.split() if word not in stopword]
    #print(s_list)
    str_ = ' '.join(s_list)   
    s.append(str_) 
print(s) 


{'a', 'is', 'of', 'the', 'in'}
['Karachi one most congested city world.', 'Karachi financial capital Pakistan.', 'Islamabad capital Pakistan.']


In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [5]:
cnt_vectorizer = CountVectorizer()
cnt_vectorizer.fit(s)
#print(cnt_vectorizer.vocabulary_)
print(cnt_vectorizer.get_feature_names())

['capital', 'city', 'congested', 'financial', 'islamabad', 'karachi', 'most', 'one', 'pakistan', 'world']


In [6]:
vec1 = cnt_vectorizer.transform(s).toarray()
print(vec1)

[[0 1 1 0 0 1 1 1 0 1]
 [1 0 0 1 0 1 0 0 1 0]
 [1 0 0 0 1 0 0 0 1 0]]


### BiGram, TriGram and Limiting Maximum Features

In [7]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
bigram_vectorizer.fit(s)
print(bigram_vectorizer.get_feature_names())

['capital', 'capital pakistan', 'city', 'city world', 'congested', 'congested city', 'financial', 'financial capital', 'islamabad', 'islamabad capital', 'karachi', 'karachi financial', 'karachi one', 'most', 'most congested', 'one', 'one most', 'pakistan', 'world']


In [9]:
vec2 = bigram_vectorizer.transform(s).toarray()
print(vec2.shape)

(3, 19)


In [10]:
cs2 = cosine_similarity(vec2)
print(cs2)

[[1.         0.11396058 0.        ]
 [0.11396058 1.         0.50709255]
 [0.         0.50709255 1.        ]]


In [11]:
cs1 = cosine_similarity(vec1)
print(cs1)

[[1.         0.20412415 0.        ]
 [0.20412415 1.         0.57735027]
 [0.         0.57735027 1.        ]]


In [17]:
#If not None, build a vocabulary that only consider the top max_features ordered by term frequency 
#across the corpus.
bigram_vectorizer2 = CountVectorizer(ngram_range=(1, 2), max_features=8)
bigram_vectorizer2.fit(s)
print(bigram_vectorizer2.get_feature_names())


['capital', 'capital pakistan', 'karachi', 'most', 'most congested', 'one', 'one most', 'pakistan']


In [18]:
vec3 = bigram_vectorizer2.transform(s).toarray()
cs3 = cosine_similarity(vec3)
print(cs3)

[[1.        0.2236068 0.       ]
 [0.2236068 1.        0.8660254]
 [0.        0.8660254 1.       ]]


## Sentiment Analysis

The subset of Amazon dataset used in this notebook can be downloaded from https://gist.github.com/kunalj101/ad1d9c58d338e20d09ff26bcc06c4235

In [13]:
#import os
#print("Current Working Directory " , os.getcwd())

In [15]:
# load the dataset
with open('DataSet/Amazon/corpus','r',encoding="utf8") as f:
    document = f.readlines()
f.close()

labels, texts = [], []
for line in document:
    content = line.split()
    label = content[0]
    labels.append(label[-1])
    texts.append(" ".join(content[1:]))

print(len(labels), len(texts))

10000 10000


In [1]:
texts[0:2]

NameError: name 'texts' is not defined

In [19]:
labels[0:7]

['2', '2', '2', '2', '2', '2', '1']

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
#from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
#from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

### Using Count Vectorizer

In [39]:
cnt_vectorizer = CountVectorizer() #Try with different number of features and n-grams max_features=5000
#cnt_vectorizer = CountVectorizer(ngram_range=(1, 3),max_features=3500)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()

In [42]:
features_nd.shape

(10000, 31627)

In [41]:
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)

In [40]:
encoder = preprocessing.LabelEncoder()
labels = encoder.fit_transform(labels)

In [35]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [36]:
y_train[0:7]

array([1, 1, 0, 0, 0, 0, 0], dtype=int64)

In [54]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)
accuracy_score(y_test,nb_pred)

0.838

In [58]:
rf = RandomForestClassifier(n_estimators=20)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
accuracy_score(y_test, rf_pred)nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)
accuracy_score(y_test,nb_pred)

0.7728

In [63]:
gb = GradientBoostingClassifier(n_estimators=5, learning_rate = 0.2)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
accuracy_score(y_test, gb_pred)


0.7212

### Using TFIDF Vectorizer

In [59]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
features = tfidf_vectorizer.fit_transform(texts)
features_nd = features.toarray()
features_nd.shape

(10000, 5000)

In [60]:
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)

In [61]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)
accuracy_score(y_test,nb_pred)

0.838

In [62]:
rf = RandomForestClassifier(n_estimators=20)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
accuracy_score(y_test, rf_pred)

0.7768

## Clustering

In [64]:
with open('headlines.txt','r',encoding="utf8") as f:
    document = f.readlines()
f.close()
document

['Pakistan will address actionable evidence if shared by Delhi, PM Khan tells India after Pulwama attack\n',
 'Roadmap for $21bn Saudi investment finalised\n',
 'Pakistani nationals ordered to leave Indian city in 48 hours: TOI report\n',
 'PM Imran offers India probe into Kashmir suicide attack\n',
 'Riyadh to invest $21b over next five years\n',
 'Pakistanis visiting India given 48-hours to leave the country\n',
 '“If India attacks, we will not think and retaliate back”: PM Imran\n',
 'Pakistan hails record Saudi investment as Crown Prince departs after first state visit\n',
 'Pakistan nationals told to leave Indian city within 48 hours\n',
 'Pakistan PM Imran Khan promises action if India shows Pulwama proof, warns against any rash move \n',
 'Saudi Arabia to invest 21 bln dollars in Pakistan in three phases\n',
 "Leave Within 48 Hours, Pak Nationals Told In Order In Rajasthan's Bikaner"]

In [65]:
cnt_vectorizer = CountVectorizer() 
features = cnt_vectorizer.fit_transform(document)
features_nd = features.toarray()
features_nd

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

In [67]:
features_nd.shape

(12, 89)

In [70]:
from sklearn.cluster import KMeans

In [71]:
km = KMeans(n_clusters =3)
km.fit(features_nd)
new_labels = km.labels_
print(new_labels)

[1 2 0 1 2 0 1 2 0 1 2 0]


In [ ]:
cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')
cluster.fit_predict(X)